# Simulations of cereal monocrops and intercrops

In [3]:
import math
import time
from random import *
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from itertools import product
from ipywidgets import interact, interactive, IntSlider, fixed

from openalea.mtg.traversal import pre_order2, pre_order2_with_filter
from openalea.plantgl.all import (Color3, Material, Vector3)
nice_green = Color3((50, 100, 0))
from oawidgets.plantgl import *

from alinea.caribu.CaribuScene import CaribuScene
from alinea.caribu.data_samples import data_path

# from archicrop
from openalea.archicrop.cereals import build_shoot
from openalea.archicrop.plant_shape import compute_leaf_area_pot_plant
from openalea.archicrop.display import display_mtg, build_scene, display_scene
from openalea.archicrop.stand import regular_plot, agronomic_plot
from openalea.archicrop.growth import thermal_time
from openalea.archicrop.light_it import compute_light_inter

# Enable plotting with PlantGL
%gui qt

## Generate potential plants from architectural parameters

In [13]:
# import os

# def create_and_store_files_with_dict(root_dir, param_names, param_combinations, generate_content):
#     """
#     Creates and organizes files using a dictionary for parameter names.

#     :param root_dir: Root directory for the files.
#     :param param_names: List of parameter names (order corresponding to the combinations).
#     :param param_combinations: List of lists, each list containing parameter values.
#     :param generate_content: Function to generate the file content based on the parameters.
#     """
    
#     if not os.path.exists(root_dir):
#         os.makedirs(root_dir)

#     for param_values in param_combinations:
#         # Associate values with names to create a dictionary of parameters
#         params = dict(zip(param_names, param_values))

#         # Build the folder path based on the parameters
#         subdir = os.path.join(root_dir, f"{param_names[0]}_{params[param_names[0]]}", 
#                                        f"{param_names[1]}_{params[param_names[1]]}")
#         os.makedirs(subdir, exist_ok=True)

#         # Filename
#         filename = f"{param_names[2]}_{params[param_names[2]]}.txt"
#         filepath = os.path.join(subdir, filename)

#         # Generate and write the content
#         content = generate_content(params)
#         with open(filepath, 'w') as f:
#             f.write(content)

# # Function to generate the file content
# def generate_content(params):
    
#     return 

# # Dictionary of parameter names
# param_names = archi_parameters_ranges.keys()

# # List of combinations
# param_combinations = archi_parameters_combinations[:10]

# # Create the files
# root_dir = "D:\PhD_Oriane\simulations_ArchiCrop\morphotypes\sorghum"
# create_and_store_files_with_dict(root_dir, param_names, param_combinations, generate_content)


# # Export all the properties defined in `g`.
# # We consider that all the properties are real numbers.

# properties = [(p, 'REAL') for p in g.property_names() if p not in ['edge_type', 'index', 'label']]
# mtg_lines = write_mtg(g, properties)

# # Write the result into a file example.mtg

# filename = 'example.mtg'
# f = open(filename, 'w')
# f.write(mtg_lines)
# f.close()

In [ ]:
# import shutil

# shutil.make_archive("output_files_compressed", 'zip', "output_files")